In [3]:
import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage
import lsst.afw.geom as afwGeom

import math

# We are going to analyse the following patches. We assume that these patches are
# all in the same tract (0)
patchList = ['1,3', '2,3']
#patchList = ['1,1', '1,3', '1,2', '1,4', '1,5', '2,1', '2,2', '2,3', '2,4', '2,5', '3,2', '3,3', '3,4']
#patchList = ['3,3', '4,3', '3,4', '4,4']

# Initialize butler
butler = dafPersist.Butler("/home/boutigny/LSST/CFHT/clusters/MACSJ2243.3-0935/output/coadd_dir_cc")
#butler = dafPersist.Butler("/home/boutigny/LSST/CFHT/clusters/CL0016+16/output/coadd_dir_cc")

In [6]:
for i, patch in enumerate(patchList) :
    
    dataId_r = {'tract':0, 'filter':'r', 'patch':patch}
    dataId_g = {'tract':0, 'filter':'g', 'patch':patch}
    dataId_i = {'tract':0, 'filter':'i', 'patch':patch}

    meas_r = butler.get('deepCoadd_meas', dataId=dataId_r)
    meas_g = butler.get('deepCoadd_meas', dataId=dataId_g)
    meas_i = butler.get('deepCoadd_meas', dataId=dataId_i)
    
    phot_r = butler.get('deepCoadd_forced_src', dataId=dataId_r)
    phot_g = butler.get('deepCoadd_forced_src', dataId=dataId_g)
    phot_i = butler.get('deepCoadd_forced_src', dataId=dataId_i)

    md_r = butler.get('deepCoadd_calexp', dataId=dataId_r)
    md_g = butler.get('deepCoadd_calexp', dataId=dataId_g)
    md_i = butler.get('deepCoadd_calexp', dataId=dataId_i)
    calib_r = md_r.getCalib()
    calib_g = md_g.getCalib()
    calib_i = md_i.getCalib()

    if i == 0 :
        schema = meas_r.getSchema()
        schema_phot = phot_r.getSchema()

        # Get keys from the measurement catalog
        # The following is not strictly necessary as one could use the get("key_name") method to access values in the
        # catalogs, but it is much more efficient to use get(key)
        fluxKey = schema_phot["modelfit_CModel_flux"].asKey()
        fluxSigmaKey = schema_phot["modelfit_CModel_fluxSigma"].asKey()
        fluxFlagKey = schema_phot["modelfit_CModel_flag"].asKey()
        extKey = schema["base_ClassificationExtendedness_value"].asKey()
        extFlagKey = schema["base_ClassificationExtendedness_flag"].asKey()
        nChildKey = schema["deblend_nChild"].asKey()
        primaryKey = schema["detect_isPrimary"].asKey()

        raKey = schema["coord_ra"].asKey()
        decKey = schema["coord_dec"].asKey()
        
        # Initialize some lists
        magR_star = []
        magG_star = []
        magI_star = []
        magR_gal = []
        magG_gal = []
        magI_gal = []
        raSrc = []
        decSrc = []
        res_i = []
        res_r = []
    
    # Loop over deblended sources in the r, g, and i deepCoadd_meas catalogs
    for i in range(len(meas_r)) :
        # Reject parents of deblended objects
        if not meas_r[i].get(primaryKey) :
            continue
        # Select stars
        if meas_r[i].get(extFlagKey):
            continue
        # Select sources which have a proper flux value in r, g and i bands
        # Notice that it would not be strictly necessary with forced photometry
        if phot_r[i].get(fluxFlagKey) or phot_g[i].get(fluxFlagKey) or phot_i[i].get(fluxFlagKey) :
            continue
        flux_r = phot_r[i].get(fluxKey)
        flux_g = phot_g[i].get(fluxKey)
        flux_i = phot_i[i].get(fluxKey)
        if flux_r <= 0. or flux_g <= 0. or flux_i <= 0. :
            continue
        fluxS_r = phot_r[i].get(fluxSigmaKey)
        fluxS_g = phot_g[i].get(fluxSigmaKey)
        fluxS_i = phot_i[i].get(fluxSigmaKey)
        if flux_r/fluxS_r < 10. or flux_g/fluxS_g < 10. or flux_i/fluxS_i < 10. :
            continue

        # Need to use a calibobject in order to convert flux to magnitude
        mag_r = calib_r.getMagnitude(flux_r)
        mag_g = calib_g.getMagnitude(flux_g)
        mag_i = calib_i.getMagnitude(flux_i)

        if meas_r[i].get(extKey) < 0.5 : 
            magR_star.append(mag_r)
            magG_star.append(mag_g)
            magI_star.append(mag_i)
        else :
            magR_gal.append(mag_r)
            magG_gal.append(mag_g)
            magI_gal.append(mag_i)

        ra = meas_r[i].get(raKey)
        dec = meas_r[i].get(decKey)
        raSrc.append(float(ra))
        decSrc.append(float(dec))

    print "Nbr. of selected sources after reading patch %s : %d"%(patch, len(magR_star)+len(magR_gal))

Nbr. of selected sources after reading patch 1,3 : 2969
Nbr. of selected sources after reading patch 2,3 : 7934


In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pylab as plt
import numpy as np

font = {'family' : 'serif',
        'color'  : 'darkred',
        'weight' : 'bold',
        'size'   : 25,
        }

plt.rcParams['axes.linewidth'] = 1.5 #set the value globally

rS = np.asarray(magR_star)
iS = np.asarray(magI_star)
gS = np.asarray(magG_star)

idxs = np.where(rS<23)

rG = np.asarray(magR_gal)
iG = np.asarray(magI_gal)
gG = np.asarray(magG_gal)

idxg = np.where(rG<22)

# color plots
fig, (ax1) = plt.subplots(ncols=1, figsize=(10, 10))
ax1.scatter(gS[idxs]-rS[idxs], rS[idxs]-iS[idxs], s=1, color='b', label="stars %d"%len(gS[idxs]))
ax1.scatter(gG[idxg]-rG[idxg], rG[idxg]-iG[idxg], s=1, color='r', label="galaxies %d"%len(gG[idxg]))
ax1.set_xlim([-0.5,2])
ax1.set_ylim([-0.5,2.5])
ax1.set_xlabel("g-r", fontsize=25)
ax1.set_ylabel("r-i", fontsize=25)
ax1.tick_params(labelsize=15)
ax1.legend(loc="upper left")

NameError: name 'magR_star' is not defined

In [1]:
print measschema.getOrderedList()

NameError: name 'schema' is not defined